-----------------------
## 1) New way of scraping  11.4.2024
-----------------------

In [8]:
from datetime import datetime
import pandas as pd
from tqdm import tqdm  
import requests 

import os
os.chdir("c:\\Users\\jirka\\Documents\\MyProjects\\Sreality")
from scraper.scraper import SrealityScraper
from db_managment.data_manager import DataManager
from utils.utils import Utilities

### My regular run

In [ ]:
scraper = SrealityScraper()
utils = Utilities()

full_datetime, date_to_save = utils.generate_timestamp()
print(full_datetime, date_to_save)

data_prodej_byty = scraper.run_scraping(timestamp=full_datetime, category_main_cb=1,category_type_cb=1)
df_data_prodej_byty = pd.DataFrame(data_prodej_byty)
utils.safe_save_csv(df_data_prodej_byty, f"data_{date_to_save}")

data_all = scraper.run_scraping(timestamp=full_datetime)
df_data_all = pd.DataFrame(data_all)
utils.safe_save_csv(df_data_all, f"data_all_{date_to_save}")

In [11]:
#test zda dobře parsuju
headers = {"User-Agent": "Mozilla/5.0"}

with requests.Session() as session:
    url = "https://www.sreality.cz/api/cs/v2/estates/1214567756"
    result =  session.request(method="GET", url=url, headers=headers).json()
    display(result)

{'text': {'name': 'Popis',
  'value': 'Prodej apartmánu 2+kk s balkónem a parkovacím místem v Horní Lipové, která je součástí obce Lipová-lázně v okr. Jeseník. Byt je v prvním patře nově postaveného bytového domu s výtahem. \r\nKompletní zařízení bytu novým nábytkem na míru včetně kuchyňské linky, vinylové podlahy v pokojích a předsíni, keramická dlažba v koupelně. Byt má ústřední topení a ohřev teplé vody zajištěné společnou plynovou kotelnou v domě. Elektřina 230 + 400 V. pro indukční varnou desku a horkovzdušnou troubu, voda z obecního vodovodního řádu. Zaveden internet a satelitní TV. Kolaudace domu proběhla koncem roku 2019.\r\nVýměra bytu včetně balkónu orientovaného na severo-východní stranu je 40,8 m2, sklep 2,5 m2, vlastní parkovací místo před objektem o výměře 12 m2. Nízké náklady na bydlení, fond oprav ve výši 7 Kč na 1 m2. \r\nJedná se o velmi pěkné klidné místo vhodné k rekreaci i bydlení v obci, která je na rozhraní Hrubého Jeseníku a Rychlebských hor. Zastávka vlaku je v

In [9]:
# Just testing count
headers = {"User-Agent": "Mozilla/5.0"}

with requests.Session() as session:
    url = "https://www.sreality.cz/api/cs/v2/estates/count"
    result =  session.request(method="GET", url=url, headers=headers).json()
    print(result)
    url_filter = f"https://www.sreality.cz/api/cs/v2/estates?category_main_cb=1&category_type_cb=1&per_page=1&page=1"
    result2 =  session.request(method="GET", url=url_filter).json()
    print(result2)

{'logged_in': False, 'result_size': 93473, '_links': {'self': {'profile': '/estates/count/doc', 'href': '/cs/v2/estates/count', 'title': 'Estate count'}}}
{'meta_description': '16960 realit v nabídce prodej bytů. Vyberte si novou nemovitost na sreality.cz s hledáním na mapě a velkými náhledy fotografií nabízených bytů.', 'result_size': 16960, '_embedded': {'estates': [{'labelsReleased': [[], ['tram', 'restaurant']], 'has_panorama': 0, 'labels': ['Tramvaj 2 min. pěšky', 'Restaurace 3 min. pěšky'], 'is_auction': False, 'labelsAll': [['personal', 'brick', 'cellar', 'partly_furnished'], ['candy_shop', 'small_shop', 'theater', 'vet', 'tavern', 'movies', 'playground', 'sightseeing', 'natural_attraction', 'bus_public_transport', 'atm', 'medic', 'school', 'train', 'kindergarten', 'drugstore', 'restaurant', 'metro', 'post_office', 'shop', 'tram', 'sports']], 'seo': {'category_main_cb': 1, 'category_sub_cb': 4, 'category_type_cb': 1, 'locality': 'praha-nove-mesto-podskalska'}, 'exclusively_at_rk

## Table managment test

In [5]:
from db_managment.data_manager import DataManager
import pandas as pd

data_manager = DataManager()

In [ ]:
data_manager.get_all_rows("estate_detail")


In [ ]:
data_manager.clear_table("estate_detail")

In [ ]:
data_manager.drop_table("estate_detail")

In [ ]:
data_manager.create_table("estate_detail")

In [6]:

#TODO: how about unit test for this?
#? should fail predictably as it breakes foreign key constraints i made:)
upload = pd.DataFrame(data = [["911", "4242", "12345000"]],
                      columns=["estate_id", "batch_id", "price"])

data_manager.insert_new_price(upload)

TypeError: DataManager.insert_new_price() missing 1 required positional argument: 'timestamp'

## Processing new offers

In [ ]:
from db_managment.data_manager import DataManager
from utils.utils import Utilities
import pandas as pd

data_manager = DataManager()
utils = Utilities()

In [ ]:
data_manager.drop_table("estate_detail")
data_manager.create_table("estate_detail")

In [ ]:
df_new = pd.read_csv("data_2024/data_20240420_0019.csv", sep=";", encoding="utf-8")
if 'Unnamed: 0' in df_new.columns:
    df_new.drop('Unnamed: 0', axis=1, inplace=True)

full_datetime, date_to_save = utils.generate_timestamp()

data_manager.process_new_estates(df_new, full_datetime)

In [3]:
for i in range(50):
    res = utils.get_location(df_new.loc[i, "latitude"], df_new.loc[i, "longitude"])
    parsed = utils.parse_location(res)
    print(parsed)

{'oblast': 'Řepy', 'město': 'Praha', 'okres': 'obvod Praha 6', 'kraj': 'Hlavní město Praha'}
{'oblast': 'Smíchov', 'město': 'Praha', 'okres': 'obvod Praha 5', 'kraj': 'Hlavní město Praha'}
{'oblast': 'Smíchov', 'město': 'Praha', 'okres': 'obvod Praha 5', 'kraj': 'Hlavní město Praha'}
{'oblast': 'Mladá Boleslav II', 'město': 'Mladá Boleslav', 'okres': 'okres Mladá Boleslav', 'kraj': 'Střední Čechy'}
{'oblast': 'Strašnice', 'město': 'Praha', 'okres': 'obvod Praha 10', 'kraj': 'Hlavní město Praha'}
{'oblast': 'Mladá Boleslav II', 'město': 'Mladá Boleslav', 'okres': 'okres Mladá Boleslav', 'kraj': 'Střední Čechy'}
{'oblast': 'Jana a Jos. Kovářů', 'město': 'Ústí nad Orlicí', 'okres': 'okres Ústí nad Orlicí', 'kraj': 'Pardubický kraj'}
{'oblast': 'Smíchov', 'město': 'Praha', 'okres': 'obvod Praha 5', 'kraj': 'Hlavní město Praha'}
{'oblast': 'Strašnice', 'město': 'Praha', 'okres': 'obvod Praha 10', 'kraj': 'Hlavní město Praha'}
{'oblast': 'Mladá Boleslav II', 'město': 'Mladá Boleslav', 'okres

worfklow new price, pokud neexistuje Estate, resp batch
- load csv s offers
- filter_new_estates - najde mi ty, jejichž code ještě není v tabulce estate_details
- přeloží to pro ně typ budovy, typ  dealu, počet pokojů - z dictionary
- před zápisem do DB vygeneruje timestamp 
- insertuje se new_batch - pokud taky neexistuje !!
- jinak se může insertovat new new_estate
- a pak klidně rovnou i new price, navázané na tuto batch a price